# LeNet 5

En este Notebook se va a implementar la red convolucional **LeNet 5** en el dataset de nubes creado anteriormente.

In [150]:
from sklearn.externals import joblib

import numpy as np
import pandas as pd
import sklearn.utils
import matplotlib.pyplot as plt

%matplotlib inline

## Formatear el dataset
Primero cargamos los datos desde el disco

In [151]:
cloud_pd = joblib.load('data/clouds.pkl')
cloud_pd = sklearn.utils.shuffle(cloud_pd)

Dividimos el dataset en sets de train y test.

In [152]:
batchsize = 32
nData = cloud_pd.shape[0]
nData_training = int(nData * 2/3)

# Batches number for training
batches = int(nData_training / batchsize)
nData_testing = nData - nData_training

print('- Number Data training: {}'.format(nData_training))
print('- Number batches: {}'.format(batches))
print('- Number Data testing: {}'.format(nData_testing))

- Number Data training: 13312
- Number batches: 416
- Number Data testing: 6656


Crear una lookup_table con las etiquetas de las nubes y generar sus indices para el dataset.

In [153]:
cloud_lookup_table, cloud_pd.cloud_type = np.unique(cloud_pd.cloud_type.values,
                                                    return_inverse=True)

print('- Clouds Lookup-table: ', cloud_lookup_table)

- Clouds Lookup-table:  ['altocumulos' 'altostratos' 'cirros' 'estratocumulos' 'sky']


In [154]:
cloud_x = cloud_pd.iloc[:nData_training, 2:]
cloud_y = cloud_pd.iloc[:nData_training, 0]

cloud_test_x = cloud_pd.iloc[nData_training:, 2:]
cloud_test_y = cloud_pd.iloc[nData_training:, 0]

Importar paquetes necesarios de **pyTorch** y convertir los datos a tensores.

In [155]:
from torch.autograd import Variable

import torch
import torch.optim
import torch.nn as nn
import torch.nn.functional as F

In [156]:
# Training
cloud_x = cloud_x.values.reshape((batches, batchsize, 
                                  32, 32, 3)).astype('uint8')
cloud_x = np.transpose(cloud_x, axes=(0, 1, 4, 2, 3))

cloud_y = cloud_y.values.reshape((batches, batchsize))

# Testing
cloud_test_x = cloud_test_x.values.reshape((cloud_test_x.shape[0],
                                            32, 32, 3)).astype('uint8')
cloud_test_x = np.transpose(cloud_test_x, axes=(0, 3, 1, 2))

cloud_test_y = cloud_test_y.values

# Convert to tensors
cloud_x = Variable(torch.Tensor(cloud_x).contiguous().cuda())
cloud_y = Variable(torch.LongTensor(cloud_y).contiguous().cuda())
cloud_test_x = Variable(torch.Tensor(cloud_test_x).contiguous().cuda())

print('- Cloud_x {}'.format(cloud_x.shape))
print('- Cloud_y {}'.format(cloud_y.shape))
print('- Cloud_test_x {}'.format(cloud_test_x.shape))
print('- Cloud_test_y {}'.format(cloud_test_y.shape))

- Cloud_x torch.Size([416, 32, 3, 32, 32])
- Cloud_y torch.Size([416, 32])
- Cloud_test_x torch.Size([6656, 3, 32, 32])
- Cloud_test_y (6656,)


En este paso se normaliza el dataset

In [157]:
cloud_x = F.normalize(cloud_x, dim=2)
cloud_test_x = F.normalize(cloud_test_x, dim=1)

## Definir la red neuronal

A partir de este punto, se va a definir la arquitecture de **LeNet-5**. Tal y como se muestra en la imagen de abajo, la red esta formada por 2 capas de convolución con un filtro de tamaño (5x5) seguidas de maxpooling (2,2) y 3 capas ocultas fully connected al final.

![Architecture of LeNet-5](https://www.researchgate.net/profile/Vladimir_Golovko3/publication/313808170/figure/fig3/AS:552880910618630@1508828489678/Architecture-of-LeNet-5.ppm)

In [226]:
class LeNet(nn.Module):
    
    def __init__(self):
        super(LeNet, self).__init__()
        # First convolutional layer, 6 channels, kernel 5x5
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        
        # Second convolutional layer, 16 channels, kernel 5x5
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        
        # Fully connected
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, len(cloud_lookup_table))
        
    def forward(self, x):
        b_size = x.size(0)

        # Max pooling (2,2)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(b_size, -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x
        
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = LeNet().cuda()
print(net)

LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=5, bias=True)
)


In [230]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.03)
loss_function = F.nll_loss
success_history = list()

import time

start = time.time()

for epoch in range(1):
    net.train()
    
    # Training
    for idx in range(cloud_x.shape[0]):
        batch = cloud_x[idx]
        target = cloud_y[idx]
        optimizer.zero_grad() # set to zero gradient buffers
        output = net.forward(batch)
        loss = F.nll_loss(output, target)
        loss.backward() # compute gradients
        optimizer.step() # do the update
        
    # Testing  
    net.eval()
    
    output_test = net(cloud_test_x).data.cpu().numpy().flatten()
    success = (cloud_test_y == output_test)
    success_history(success.sum() / success.shape[0])
    
    print('[Epoch {}] Train loss: {} Test accuracy: {}'.format(epoch, loss, success_history[idx]))
    
end = time.time()
print('\n- Time: ', end - start)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


AttributeError: 'bool' object has no attribute 'sum'

In [120]:
cloud_test_y.shape

(6656,)

In [137]:
cloud_x[0,0].shape
cloud_test_x.shape

torch.Size([6656, 3, 32, 32])

In [168]:
Variable(torch.Tensor(cloud_test_x[0])).contiguous().cuda().shape

TypeError: expected torch.FloatTensor (got torch.cuda.FloatTensor)

In [163]:
x = Variable(torch.zeros(1), requires_grad=True)
out = x * 3
out.backward()
x.grad

tensor([ 3.])

In [228]:
net.train()
optimizer.zero_grad()
print(net.forward(cloud_x[0]))

tensor(1.00000e-02 *
       [[-5.1123, -6.0579,  5.8070, -9.0513,  4.2733],
        [-5.2435, -6.0656,  5.7010, -9.1269,  4.3037],
        [-5.2144, -6.0596,  5.7415, -9.1092,  4.2850],
        [-5.0466, -6.0473,  5.8201, -8.9950,  4.2511],
        [-5.2361, -6.0557,  5.7401, -9.1382,  4.3055],
        [-5.2351, -6.0533,  5.7310, -9.1038,  4.3060],
        [-5.0760, -6.0475,  5.8356, -9.0294,  4.2802],
        [-5.2011, -6.0698,  5.7197, -9.0727,  4.3128],
        [-5.2533, -6.0663,  5.6946, -9.1282,  4.2904],
        [-5.0118, -6.0950,  5.8153, -8.9499,  4.3653],
        [-5.0158, -6.0425,  5.9046, -9.0361,  4.2516],
        [-5.8907, -6.1189,  5.6820, -9.2556,  4.7412],
        [-5.0767, -6.0626,  5.8173, -9.0233,  4.2645],
        [-5.2002, -6.0475,  5.7101, -9.0233,  4.3545],
        [-5.2375, -6.0552,  5.7408, -9.1495,  4.2945],
        [-5.0100, -6.0462,  5.9109, -9.0138,  4.2512],
        [-5.0436, -6.0451,  5.8456, -9.0065,  4.2508],
        [-4.9822, -6.0431,  5.9181, -9.0130,